In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from tensorflow.keras.layers import TimeDistributed, Flatten
from tensorflow.keras.layers import LSTM,RepeatVector
from tensorflow.keras.layers import GlobalMaxPooling1D, RepeatVector

# 加载数据
data = pd.read_csv('stock_data.csv')  # 确保文件路径是正确的
data.columns = ['price', 'vol']

# 数据缩放
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

def generator(data, lookback=10, delay=100, batch_size=32):  # 将delay设置为100
    max_index = len(data) - delay - 1
    i = lookback
    while 1:
        if i + delay >= max_index:
            i = lookback
        samples = np.zeros((batch_size, lookback, data.shape[-1]))
        targets = np.zeros((batch_size, delay, data.shape[-1]))  # Change targets to be 'delay' length sequences
        for j in range(batch_size):
            if i + delay >= max_index:
                i = lookback
            rows = np.arange(i - lookback, i)
            samples[j] = data[rows]
            targets[j] = data[i: i + delay]  # Change targets to be 'delay' length sequences
            i += 1
        yield samples, targets

lookback = 1000
delay = 50  # Set delay to 100 for predicting 100 steps ahead
train_gen = generator(data_scaled, lookback=lookback, delay=delay)

model_path = 'stock_model.h5'  # 确保路径是正确的

if os.path.exists(model_path):
    # 加载模型
    model = load_model(model_path)
else:
    # 定义模型
    inputs = Input(shape=(lookback, data.shape[-1]))

    # Encoder
    encoded = LSTM(64, return_sequences=False)(inputs)

    # Decoder
    decoded = RepeatVector(delay)(encoded)
    decoded = LSTM(64, return_sequences=True)(decoded)

    outputs = TimeDistributed(Dense(data.shape[-1]))(decoded)  # 用TimeDistributed包装Dense层，以适应新的目标序列长度
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(), loss='mse')
    model.fit(train_gen, epochs=60, steps_per_epoch=30)
    
    # 保存模型
    model.save(model_path)


Epoch 1/60


2023-06-29 14:30:54.972088: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


30/30 [==============================] - 7s 170ms/step - loss: 0.7051
Epoch 2/60
30/30 [==============================] - 5s 168ms/step - loss: 0.4128
Epoch 3/60
30/30 [==============================] - 5s 169ms/step - loss: 0.3442
Epoch 4/60
30/30 [==============================] - 5s 169ms/step - loss: 0.6028
Epoch 5/60
30/30 [==============================] - 5s 169ms/step - loss: 0.5109
Epoch 6/60
30/30 [==============================] - 5s 170ms/step - loss: 0.2320
Epoch 7/60
30/30 [==============================] - 5s 175ms/step - loss: 0.2431
Epoch 8/60
30/30 [==============================] - 5s 163ms/step - loss: 0.9515
Epoch 9/60
30/30 [==============================] - 5s 168ms/step - loss: 0.8507
Epoch 10/60
30/30 [==============================] - 5s 169ms/step - loss: 0.8604
Epoch 11/60
30/30 [==============================] - 5s 173ms/step - loss: 0.2122
Epoch 12/60
30/30 [==============================] - 5s 169ms/step - loss: 0.2733
Epoch 13/60
30/30 [=================

/Users/zhangwenbo/miniforge3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
# 对未来100个价格数据进行预测
future = 50
data_to_predict = data_scaled[-lookback:]  # 最后lookback个数据点

# 加载模型
model = load_model(model_path)

# 将输入形状改为（samples，timesteps，features）
samples = np.reshape(data_to_predict, (1, lookback, data.shape[-1]))

# 使用模型进行预测
predictions = model.predict(samples)

# 将预测结果形状改为（timesteps，features）
predicted_data = np.reshape(predictions, (delay, data.shape[-1]))

# 对预测结果进行反归一化
predicted_data = scaler.inverse_transform(predicted_data)

# 保存预测结果到csv文件
predicted_df = pd.DataFrame(predicted_data, columns=['predicted_price', 'predicted_vol'])
predicted_df.to_csv('predicted_data_encoderdecoder.csv', index=False)


1/1 [==============================] - 0s 378ms/step
